In [1]:
import pandas as pd 

# reead segments.csv
segments = pd.read_csv('segments.csv')

relevant_companies = pd.read_csv("companies_with_ren_segments_and_data.csv")

In [2]:
#get the companies in segments that have values in relevant companies
segments_relevant = segments[segments['Instrument'].isin(relevant_companies['Instrument'])]

#group by instrument and count the number of segment, and find average number of segments
segments_relevant_grouped = segments_relevant.groupby('Instrument').count()
segments_relevant_grouped['Segment Name'].mean()


7.2

In [3]:
import pandas as pd

ren_data = {}
for i in range(10): 
    path = rf"ren_alternative\ren_alternative_for_time_-{i}y.csv"
    ren_data[i] = pd.read_csv(path)

means = []
for i in range(10): 
    #find average "Company Market Cap" for each year 
    means.append(ren_data[i]['Company Market Cap'].mean())

#find the average of the averages
sum(means)/len(means)
means

[13774499671.214067,
 14601597311.617569,
 12603388345.321226,
 13586404424.828356,
 11946619036.861475,
 13201520251.737856,
 11262158541.66706,
 10262371042.636497,
 12621307465.650309,
 12413388056.03608]

In [4]:
ren_data[i].columns

Index(['Instrument', 'Revenue', 'Price Close',
       'Earnings Per Share - SmartEstimate®',
       'Earnings Per Share - SmartEstimate®.1',
       'Earnings Per Share - SmartEstimate®.2', 'Return On Equity - Actual',
       'Company Market Cap', 'Common Shares - Outstanding - Total',
       'Total Assets', 'Interest Expense', 'Total Liabilities',
       'Dividend Payout Ratio - %', 'Interest Bearing Liabilities - Total',
       'Shareholders Equity - Common', 'Total Capital', 'Total Debt',
       'Long Term Growth - Mean', 'Return on Average Common Equity - %, TTM',
       'TRBC Economic Sector Name', 'TRBC Business Sector Name',
       'TRBC Industry Group Name', 'TRBC Industry Name', 'TRBC Activity Name',
       'TRBC Economic Sector Code', 'TRBC Business Sector Code',
       'TRBC Industry Group Code', 'TRBC Industry Code', 'TRBC Activity Code',
       'Depreciation Depletion & Amortization - Total', 'WACC Tax Rate, (%)',
       'Earnings before Interest & Taxes (EBIT)', 'Country o

In [5]:
import time
import eikon as ek
def fetch_data_in_batches(tickers, batch_size, file_output_name, year=None):
    if year is None:
        year = "FY0"
    else:
        year = "FY" + str(year)

    def get_data_batch(batch):
        try:
            data, _ = ek.get_data(
                batch,
                fields=[
                    f'TR.F.BUSTOTREVBIZACTIV(SDate={year}).segmentName'
                ]
            )
            #add a column with the year
            data["year"] = year
            return data  # Return only the DataFrame
        except Exception as e:
            print(f"Failed to fetch data for batch: {batch}. Error: {e}")
            return None  # Return None in case of failure
        
    all_data_frames = []
    batches_to_process = [tickers[i:i + batch_size] for i in range(0, len(tickers), batch_size)]
    failed = []

    while batches_to_process:
        print("Number of batches to process:", len(batches_to_process)+1)
        new_failed = []
        for batch in batches_to_process:
            print("Getting data for batch number:", batches_to_process.index(batch)+1, "of", len(batches_to_process)+1)
            time.sleep(0.2)
            print("Processing batch")
            batch_data = get_data_batch(batch)
            
            if batch_data is not None:
                print("batch successful")
                all_data_frames.append(batch_data)
            else:
                print("batch failed")
                new_failed.append(batch)

        # Update the list of batches to process
        batches_to_process = new_failed

        # Remember the failed batches
        failed.extend(new_failed)

    # Concatenate all the individual batch data frames into one
    final_data_frame = pd.concat(all_data_frames, ignore_index=True) if all_data_frames else pd.DataFrame()

    # Save the data to a file (adjust the path and file name as needed)
    final_data_frame.to_csv(file_output_name, index=False)
    
    return final_data_frame, failed

In [6]:
import eikon as ek 
ek.set_app_key("5f62315c837645a78ff033f35644b3ba3ef5753a")
for i in range(10): 
    success, fail = fetch_data_in_batches(list(relevant_companies['Instrument']), 100, rf"segment_numbers\ren_segment_numbers_for_time_-{-i}y.csv", -i)
    segments_relevant_grouped = success.groupby('Instrument').count()
    print(segments_relevant_grouped['Segment Name'].mean())

Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch successful
Getting data for batch number: 2 of 3
Processing batch
batch successful
4.984
Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch successful
Getting data for batch number: 2 of 3
Processing batch
batch successful
5.04
Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch successful
Getting data for batch number: 2 of 3
Processing batch
batch successful
4.96
Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch successful
Getting data for batch number: 2 of 3
Processing batch
batch successful
4.776
Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch successful
Getting data for batch number: 2 of 3
Processing batch
batch successful
4.736
Number of batches to process: 3
Getting data for batch number: 1 of 3
Processing batch
batch